[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1u2_zKqrYyxVpwNTflTxl36UXbgPtvoN9?usp=sharing)



# Imports

In [ ]:
!pip install transformers

In [ ]:
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 9.1 MB/s eta 0:00:00


In [ ]:
!wget https://www.dropbox.com/s/neb74j04nfxay14/poetry_keywords.csv?dl=0 -O real_poetry_keywords.csv

In [ ]:
dir_path = ''

In [ ]:
import string
import re
import pickle

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine

import numpy as np
from time import sleep
from IPython.display import clear_output

/home/revolt/anaconda3/envs/poetry_gpt3_large/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from tqdm import tqdm

# get generated and real data

скачиваем

## real


In [ ]:
import pandas as pd

In [ ]:
real_poetry_df = pd.read_csv('../data/poetry_keywords.csv')

In [ ]:
real_poetry_df.shape

(7755, 3)

## generated

In [ ]:
def load_poetry(path):
  with open(path, 'rb') as f:
    generated_poetry = pickle.load(f)
  return generated_poetry

### Large one-to-many


In [ ]:
with open('../generated/key2poetry/large_all_poets.pkl', 'rb') as f:
    generated_poetry_large = pickle.load(f)

### Medium one-to-many

In [ ]:
with open('../generated/key2poetry/medium_all_poets.pkl', 'rb') as f:
    generated_poetry_medium = pickle.load(f)

### One-to-one

In [ ]:
import os

In [ ]:
poetry_all = {}
for poet in ['esenin', 'pushkin', 'mayak', 'blok', 'tyutchev']:
  poetry = load_poetry(f'../generated/image2poetry/vit_rugpt2_{poet}.pkl')
  for author in poetry:
    poetry_all[author] = poetry[author]

In [ ]:
poetry_all.keys()

dict_keys(['Пушкин', 'Тютчев', 'Есенин', 'Маяковский', 'Блок'])

### Image one-to many

In [ ]:
generated_fp32 =  load_poetry(f'../generated/image2poetry/vit_rugpt2_f32.pkl')
generated_fp16 =  load_poetry(f'../generated/image2poetry/vit_rugpt2_fp16.pkl')

In [ ]:
generated_ft = load_poetry('../generated/image2poetry/vit_rugpt2_ft.pkl')

In [ ]:
generated_ft.keys()

dict_keys(['Пушкин', 'Есенин', 'Тютчев', 'Блок', 'Маяковский'])

## from .pkl to dataframe

In [ ]:
poets = real_poetry_df['author'].unique()

In [ ]:
def from_pkl_to_df(data_pkl: pd.DataFrame):  
  generated_df = pd.DataFrame()

  for poet in tqdm(poets):
    new_df = pd.DataFrame()
    poetry = []
    keywords = []
    
    for line in data_pkl[poet]:
      poetry.append(line.split('Поэзия:')[-1])
      keywords.append(line.split('Поэзия:')[0].split('Ключевые слова:')[-1].replace('\n',''))

    new_df['author'] = [poet] * len(data_pkl[poet])
    new_df['text'] = poetry
    new_df['keywords'] = keywords
    generated_df = pd.concat([generated_df, new_df])
  return generated_df

generated_medium_df = from_pkl_to_df(generated_poetry_medium)
generated_large_df = from_pkl_to_df(generated_poetry_large)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 346.12it/s]


In [ ]:
def from_pkl_to_df_wo_key(data_pkl: pd.DataFrame):  
  generated_df = pd.DataFrame()

  for poet in tqdm(poets):
    new_df = pd.DataFrame()
    new_df['author'] = [poet] * len(data_pkl[poet])
    new_df['text'] = data_pkl[poet]

    generated_df = pd.concat([generated_df, new_df])
  return generated_df

generated_O2O = from_pkl_to_df_wo_key(poetry_all)

100%|███████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 395.88it/s]


In [ ]:
generated_O2O

,author,text
0,Маяковский,"!\nВовсю балаганит,\nсмеется,\nулыбаясь.\nЭто ..."
1,Маяковский,и снова\n взлетает на дыбы.\nИ снова\n ...
2,Маяковский,",\nа обратно —\nручкой,\nкуда надо.\nИ так — г..."
3,Маяковский,"-Ты что, не рада?\nТы что, не рада?\nКак же эт..."
4,Маяковский,":-)\nА вообще, мне нравится «Араби».\nВкусная,..."
...,...,...
2202,Пушкин,"(Мф.24:24–28)\nПошли мне жребий ваш,\nЗаране ..."
2203,Пушкин,", О вы, которых трепетали\nЕвропы сильны племе..."
2204,Пушкин,":\nИ тем небо умножает,\nИ мне блаженство внуш..."
2205,Пушкин,", О нет, мне жизнь не надоела,\nЯ жить люблю, ..."


## Preprocess data

In [ ]:
POETS = list(real_poetry_df['author'].unique())

делаем небольшую чистку для формирования эмбеддингов

In [ ]:
def clean_line(text):
  # Convert to lowercase

  text = text.lower()
    
  # Remove punctuation
  text = text.translate(str.maketrans("", "", string.punctuation))
  
  # Remove numbers
  text = re.sub(r'\d+', '', text)
  
  # # Remove unknown symbols (keep only alphabets and spaces)
  # text = re.sub(r'[^a-zа-яё\s]', '', text)

  # Strip leading and trailing whitespace
  text = text.strip()
  
  # Replace multiple consecutive spaces with a single space
  text = re.sub(r'\s+', ' ', text)
  
  return text

преобразуем в лист, содержащий построчно все произведения

In [ ]:
def get_poetry_lines(data: list):
  poetry_lines = []
  for poetry in data:
    lines = poetry.split('\n')
    for line in lines:
      if len(line.split()) < 30:
        # line = clean_line(line) # очистка
        poetry_lines.append(line)
  return poetry_lines

In [ ]:
poetry_lines = {poet:{} for poet in POETS}

for poet in POETS:
  # real
  real_poetry_list = real_poetry_df[real_poetry_df['author']==poet]['text'].to_list()
  print(len(real_poetry_list))
  poetry_lines[poet]['real'] = get_poetry_lines(real_poetry_list)

  # image2one_poet
  generated_O2O_list = generated_O2O[generated_O2O['author'] == poet]['text'].to_list()
  print(len(generated_O2O_list))
  poetry_lines[poet]['generated_medium_O2O'] = get_poetry_lines(generated_O2O_list)

  #image to many poets
  poetry_lines[poet]['generated_fp16'] = get_poetry_lines(generated_fp16[poet])
  poetry_lines[poet]['generated_fp32'] = get_poetry_lines(generated_fp32[poet])
  poetry_lines[poet]['generated_ft'] = get_poetry_lines(generated_ft[poet])
  
  # keywords to many
  print(len(generated_poetry_large[poet]))
  generated_large_list = generated_large_df[generated_large_df['author'] == poet]['text'].to_list()
  generated_medium_list = generated_medium_df[generated_medium_df['author'] == poet]['text'].to_list()
  poetry_lines[poet]['generated_large_O2M'] = get_poetry_lines(generated_large_list)
  poetry_lines[poet]['generated_medium_O2M'] = get_poetry_lines(generated_medium_list)

# Bert to form embeddings by all poetry

https://huggingface.co/docs/transformers/model_doc/bert

Load the pretrained SBERT model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_name = 'sberbank-ai/sbert_large_mt_nlu_ru'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

Define a function to generate embeddings:

In [ ]:
max_length = tokenizer.model_max_length

In [ ]:
def get_embeddings(texts, batch_size=32):
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        clear_output()

        print(f'Batch {i/batch_size}/{len(texts)/batch_size}')
        batch_texts = texts[i:i+batch_size]
        tokens = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')
        
        # Move the input tensors to the device
        tokens = {key: value.to(device) for key, value in tokens.items()}
        
        with torch.no_grad():
            embeddings = model(**tokens).last_hidden_state[:, 0, :]
            # Move the embeddings back to the CPU and convert to a NumPy array
            embeddings = embeddings.cpu().numpy()
        
        all_embeddings.extend(embeddings)

        # sleep(0.5)
    
    return np.array(all_embeddings)

Compute the embeddings for both sets of texts:

In [ ]:
embeddings_dict = {poet:{} for poet in POETS}

for poet in POETS:
  print(poet)
  embeddings_dict[poet]['generated_large_O2M'] = get_embeddings(poetry_lines[poet]['generated_large_O2M'])
  embeddings_dict[poet]['generated_medium_O2M'] = get_embeddings(poetry_lines[poet]['generated_medium_O2M'])
  embeddings_dict[poet]['real'] = get_embeddings(poetry_lines[poet]['real'])
  embeddings_dict[poet]['generated_medium_O2O'] = get_embeddings(poetry_lines[poet]['generated_medium_O2O'])
  embeddings_dict[poet]['generated_fp16'] = get_embeddings(poetry_lines[poet]['generated_fp16'])
  embeddings_dict[poet]['generated_fp32'] = get_embeddings(poetry_lines[poet]['generated_fp32'])
  embeddings_dict[poet]['generated_ft'] = get_embeddings(poetry_lines[poet]['generated_ft'])

Batch 63.0/698.25
Batch 63.0/698.25


### save embeddings

In [ ]:
def save_embeddings(embeddings_dict: dict, file_name):
  with open(file_name, 'wb') as f:
    pickle.dump(embeddings_dict, f)

In [ ]:
save_embeddings(embeddings_dict, './embeddings_poetry.pkl')